In [8]:
import numpy as np
np.random.seed(1)
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
import pandas as pd
np.random.seed(1)
#
df1 = pd.read_csv(r"D:\Learn\BJ13_M32x32_T30_In.csv")
df2 = pd.read_csv(r"D:\Learn\BJ14_M32x32_T30_In.csv")
df3 = pd.read_csv(r"D:\Learn\BJ15_M32x32_T30_In.csv")
df4 = pd.read_csv(r"D:\Learn\BJ16_M32x32_T30_In.csv")

array1 = df1.to_numpy()
array2 = df2.to_numpy()
array3 = df3.to_numpy()
array4 = df4.to_numpy()

max_rows = max(array1.shape[0], array2.shape[0], array3.shape[0], array4.shape[0])
max_cols = max(array1.shape[1], array2.shape[1], array3.shape[1], array4.shape[1])

def pad_array(array, shape):
    padded = np.zeros(shape)
    padded[:array.shape[0], :array.shape[1]] = array
    return padded

array1_padded = pad_array(array1, (max_rows, max_cols))
array2_padded = pad_array(array2, (max_rows, max_cols))
array3_padded = pad_array(array3, (max_rows, max_cols))
array4_padded = pad_array(array4, (max_rows, max_cols))

X0 = np.stack((array1_padded, array2_padded, array3_padded, array4_padded), axis=0)
print(X0.shape)
print(type(X0))

(4, 7219, 1024)
<class 'numpy.ndarray'>


In [9]:
def dell(array1,missing_rate):
    array2 = array1 * np.round(np.random.rand(array1.shape[0], array1.shape[1]) + 0.5 - missing_rate)
    return array2
array1 = dell(array1,0.5)
array2 = dell(array2,0.5)
array3 = dell(array3,0.5)
array4 = dell(array4,0.5)

In [10]:
array1_dell = pad_array(array1, (max_rows, max_cols))
array2_dell = pad_array(array2, (max_rows, max_cols))
array3_dell = pad_array(array3, (max_rows, max_cols))
array4_dell = pad_array(array4, (max_rows, max_cols))
X = np.stack((array1_padded, array2_padded, array3_padded, array4_padded), axis=0)
print(X.shape)
print(type(X))

(4, 7219, 1024)
<class 'numpy.ndarray'>


In [3]:
import tensorly as tl
from tensorly.decomposition import tucker
def Res():
    return 0
def fold(unfolded, shape, mode):
    full_shape = np.array(shape)
    full_shape[mode], full_shape[0] = full_shape[0], full_shape[mode]
    return np.moveaxis(np.reshape(unfolded, full_shape), 0, mode)
def unfold(z,rank):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1))
def update_cg(var, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return var, r, q, rnew
def ell_G(G, S, R, M, X, K,n):
    return K@G@S@R@M@S.T@R.T@M.T@+n@G
def conj_grad_G(G, S, R, M, X, K, n,maxiter = 5):
    rank, dim1 = G.shape
    r = np.reshape(K@X@S.T@R.T@M.T-ell_G(G, S, R, M, X, K,n) ,-1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_G(G, S, R, M, X, K,H,n), -1, order = 'F')
        G, r, q, rold = update_cg(G, r, q, Aq, rold)
    return np.reshape(G, (rank, dim1), order = 'F')
def conj_grad_S_R_M(G, S, R, M, X, K,Q1,H,a,b,n ,r1,maxiter = 5):
    def ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        return (K@S@G@(np.kron(M,R)).T@(np.kron(M,R))@G.T
                +r1*(S-A*Q@S-b*(np.kron(Q1,H))@S)@(np.eye(Q1.shape[0])-a*Q1.T-b*(np.kron(Q1,H)).T)+n*S)
    def ell_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        return (K@R@G@(np.kron(M,S)).T@(np.kron(M,S))@G.T
                +r1*(R-A*Q@R-b*(np.kron(Q1,H))@R)@(np.eye(Q1.shape[0])-a*Q1.T-b*(np.kron(Q1,H)).T)+n*R)
    def ell_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        return (K@M@G@(np.kron(R,S)).T@(np.kron(R,S))@G.T
                +r1*(M-A*Q@M-b*(np.kron(Q1,H))@M)@(np.eye(Q1.shape[0])-a*Q1.T-b*(np.kron(Q1,H)).T)+n*M)

    def con_grad_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        X=unfold(X,1)
        G=unfold(G,1)
        K=unfold(K,1)
        rank, dim1 = S.shape
        r=np.reshape(K@X@(np.kron(M,R))@G.T-ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            S, r, q, rold = update_cg(S, r, q, Aq, rold)        
        return np.reshape(S, (rank, dim1), order = 'F')
    
    def con_grad_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        X=unfold(X,2)
        G=unfold(G,2)
        K=unfold(K,2)
        rank, dim1 = R.shape
        r=np.reshape(K@X@(np.kron(M,S))@G.T-ell_R(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            R, r, q, rold = update_cg(R, r, q, Aq, rold)        
        return np.reshape(R, (rank, dim1), order = 'F')

    def con_grad_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        X=unfold(X,3)
        G=unfold(G,3)
        K=unfold(K,3)
        rank, dim1 = M.shape
        r=np.reshape(K@X@(np.kron(R,S))@G.T-ell_M(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            M, r, q, rold = update_cg(M, r, q, Aq, rold)        
        return np.reshape(M, (rank, dim1), order = 'F')
    S=con_grad_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)
                                                                                           
                                                                                           
    return S,con_grad_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5),con_grad_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)

                                                                                           
                                                                                           
                                                                                           
def conj_grad_Q_H(Q, H, Z, a, b):
    def conj_grad_Q(Q, H, Z, a, b):
        return np.reshape(M, (rank, dim1), order = 'F')
    def conj_grad_H(Q, H, Z, a, b):
        return np.reshape(M, (rank, dim1), order = 'F')
    return 0           
def conj_grad_a_b(Q, H, Z, a, b):
    return 0
def Star(X0,X,rank,thers,maxiter):
    #Create K
    K = np.zeros(tensor1.shape) 
    K[(X0 != 0) & (X == 0)] = 1
    
    #Initialize G S R M Q H A B randomly
    G = np.random.rand(rank, rank, rank)
    S = np.random.rand(X.shape[0], rank)
    R = np.random.rand(X.shape[1], rank)
    M = np.random.rand(X.shape[2], rank)
    Q = np.random.rand(X.shape[2], X.shape[2])
    H = np.random.rand(X.shape[1], X.shape[1])
    a = np.random.uniform(0, 1)
    b = np.random.uniform(0, 1)
    #iter
    for iter in range(maxiter):
        for k in range(3):
            Gshape=G.shape
            Kshape=K.shape
            Xshape=X.shape
            G=unfold(G,k)
            X=unfold(X,k)
            K=unfold(K,k)
            G = conj_grad_G(G, S, R, M, X, K)
            G = fold(G, Gshape, k)
            K = fold(K, Gshape, k)
            X = fold(X, Gshape, k)
            S, R, M = conj_grad_S_R_M(G, S, R, M, X, K,Q,H,a,b,n ,r)
            Q, H = conj_grad_Q_H(Q, H, Z, a, b)
            a, b = conj_grad_a_b(Q, H, Z, a, b)
        if Res()<thers:
            break
    return 0

In [ ]:
a=1
    #rank=[50,50,50]
    #core, factors = tucker(X, rank)
    #G = core
    #S = factors[0]
    #R = factors[1]
    #M = factors[2]